In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb

In [2]:
from ipywidgets import widgets
from IPython.display import display

In [3]:
def stock(arg):
    #get all available tickers
    tickers = list(data.get_iex_symbols()['symbol'])
    stock=[]
    global start
    global end
    global n
    #get the start date
    start_date=start.value
    #get the end data
    end_date=end.value
    #get N value from 
    N=int(n.value)
    avg_price_book_ratio=pd.Series(dtype=np.float64)
    if start_date!='' and end_date!='':
        try:
            for i in tickers:
                try:
                    #find the book value
                    book=data.get_quote_yahoo(i)['bookValue'][i]
                    stock.append(data.DataReader(i,'yahoo',start_date,end_date))
                    #find N-average value
                    N_average=stock[-1]['Close'].sort_values().iloc[0:N].mean()
                    
                    #find the N-average price to book ratio
                    avg_price_book_ratio[i]=N_average/book
                    
                except:
                    del tickers[tickers.index(i)]
                    
            stock_data=pd.concat(stock,keys=tickers,axis=1)
            stock_data.columns.names = ['Ticker','Stock Info']
            #find closing price of the start date
            start=stock_data.xs('Close',level='Stock Info',axis=1).iloc[0]
            #find closing price of end date
            end=stock_data.xs('Close',level='Stock Info',axis=1).iloc[-1]
            #calculate dividend from the data
            dividend=stock_data.xs('Close',level='Stock Info',axis=1)-stock_data.xs('Adj Close',level='Stock Info',axis=1)
            #find total dividend
            total_dividend=dividend.sum()
            #find total years
            years=set(map(lambda x:x.year,stock_data.index))
            #find compound annual growth rate
            CAGR=((end+total_dividend/start)**(1/len(years)))-1
            CAGR=CAGR.map(lambda x:'{:.3f}%'.format(x))
            print('Compound Annual Growth Rate')
            print(CAGR)
            print('N-year average price to book ratio')
            print(avg_price_book_ratio)
        except:
            print('please enter correct date format')
    

In [4]:
button=widgets.Button(description='Refresh')
start=widgets.Text()
end=widgets.Text()
n=widgets.Text()

button.on_click(stock)
print('Start Date')
display(start)
print('End Date')
display(end)
print('N')
display(n)
display(button)

Start Date


Text(value='')

End Date


Text(value='')

N


Text(value='')

Button(description='Refresh', style=ButtonStyle())